## Como indagar o Gemini através de uma chamada de webservice

  1. Vá no site to gemini e obtenha
     * URL de chamada
     * API-KEY
     * como chamar o webservice
  2. Veja quais os modelos de LLMs treinadas da Google (Gemini e Gemma)
  3. Escolha um dos modelos
  4. Crie uma questão a ser enviada
  5. Gere a chamada para o Web Service
  6. Chame o Web Service
  7. Se a resposta for erro: avise que deu erro.
  8. Processe a resposta e armazene em 'algum lugar'


## How to query Gemini through a webservice call



### Qual meu Python?

In [ ]:
from platform import python_version
print(python_version())

### Abrindo as bibliotecas (pacotes) necessários

In [37]:
import os, sys, json, yaml
from os.path import exists
from os.path import join as osjoin
from typing import Optional, Iterable, Set, Tuple, Any, List

### Obtendo email e keypass

   Nunca mostre seu email e a chave de acesso remoto do Google AI

In [21]:
with open('params.yml', 'r') as file:
    dic_yml = yaml.safe_load(file)

In [22]:
email = dic_yml['email']
API_KEY = dic_yml['API_KEY']

In [24]:
email[:3]

'fla'

In [25]:
API_KEY[:5]

'AIzaS'

### Defaults Gemini

In [49]:
# como indagar o web service:
#   falta o modelo e a API_KEY
gemini_URL0 = "https://generativelanguage.googleapis.com/v1beta/models/%s-latest:generateContent?key=%s"

gemini_models = ['gemini-1.0-pro', 'gemini-1.5-pro', 
                 'gemini-1.5-flash-8b', 'gemini-1.5-flash', 
                 'gemini-2.0-flash-exp', 
                 'gemma-2-2b-it', 'gemma-2-9b-it', 'gemma-2-27b-it']

### Definindo o endereço do Gemini (Google AI)

In [50]:
gemini_model = gemini_models[3]
gemini_model

'gemini-1.5-flash'

In [51]:
gemini_URL = gemini_URL0%(gemini_model, API_KEY)
gemini_URL[:-30]

'https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key=AIzaSyA1Z'

In [52]:
def run_curl_gemini(question:str, temperature:float=0.1, topK:int=50, 
                    topP:float=0.10, maxOutputTokens:int=4096, stopSequences:List=[], 
                    dir_gemini:str='./', echo:bool=False, verbose:bool=False) -> List:

    question = question.replace('"','').replace("'","").replace("\n"," ")

    fname_json = f"{dir_gemini}/response.json"
    fname_cmd  = f"{dir_gemini}/run_curl.sh"

    curl_msg =  "curl -H 'Content-Type: application/json' -d " + \
                "'{\"contents\":[{\"parts\":[{\"text\":\"{" + question + "}\"}]}], " + \
                "\"generationConfig\":{" +  \
               f"\"temperature\":{temperature}, \"topK\":{topK},\"topP\":{topP}, " + \
               f"\"maxOutputTokens\":{maxOutputTokens},\"stopSequences\":{str(stopSequences)}" +  \
                "}" +  \
                "}' -X POST '{" + gemini_URL + "}' --silent >> " + fname_json

    if echo:
        print(">>> curl_msg")
        print(curl_msg)
        print("")
        return

    if os.path.exists(fname_json):
        os.remove(fname_json)

    # write the command line
    try:
        write_txt(curl_msg, fname_cmd)
        os.system(f"chmod +x '{fname_cmd}'")
    except ValueError:
        s_error = f"Error: writing '{ValueError}' in 'run_curl.sh' in run_curl_gemini()"
        print(s_error)
        return []

    response = None; dic={}

    try:
        os.system(f"{fname_cmd}")

        with open(fname_json) as f:
            dic = json.load(f)

    except ValueError:
        dic = None
        s_error = f"Error: calling gemini '{ValueError}'"
        if verbose: print(s_error)
        return []

    try:
        list_candidates = dic['candidates']
    except:
        error_code = dic['error']['code']
        error_msg  = dic['error']['message']
        s_error = f"Error: calling gemini '{error_code}: {error_msg}'"
        if verbose:
            print(s_error)
            print("---------------------")
            print(curl_msg)
            print("---------------------")
        list_candidates = []

    return list_candidates


def response_candidate(list_candidates:List, n_candidate:int=0, verbose:bool=False) -> str:

    try:
        parts = list_candidates[n_candidate]['content']['parts']

        text = ''
        for i in range(len(parts)):
            text += parts[i]['text'] + '\n'
    except:
        s_error = "Error in obtaining text in response (list_candidates)"
        if verbose: print(s_error, end=' ')
        return None

    return text

        
def write_txt(text, filename, path="./", to_append=False, verbose=False): # encondig = 'utf-8',
	if not exists(path):
		os.mkdir(path)

	filename = osjoin(path, filename)
	try:
		ret = True
		if to_append:
			h = open(filename, mode = "a+")
		else:
			h = open(filename, mode = "w")
		h.write(text)
	except ValueError:
		print(f"Error '{ValueError}', writing in '{filename}'")
		ret = False
	finally:
		h.close()

	if not ret:
		return False

	if verbose: print(f"File saved: {filename}")
	return True


### Definindo a questão

In [53]:
question = \
"""Answer in the first line Yes, Possible, Low evidence or No, and explain: 
is the cytokine storm a fundamental cause of COVID-19 deaths? 
Context: COVID-19 was a pandemic that began in late 2019 and affected the entire planet,
killing more than 6 million people. The cytokine storm is the massive release of 
inflammatory interleukins, mainly IL6, in severe cases of COVID-19."""

question = question.replace('\n','')

print(question)

Answer in the first line Yes, Possible, Low evidence or No, and explain: is the cytokine storm a fundamental cause of COVID-19 deaths? Context: COVID-19 was a pandemic that began in late 2019 and affected the entire planet,killing more than 6 million people. The cytokine storm is the massive release of inflammatory interleukins, mainly IL6, in severe cases of COVID-19.


In [54]:
temperature=0.1
topK=50
topP=0.10
maxOutputTokens=4096
stopSequences=[]

echo=True
verbose=True

list_candidates = run_curl_gemini(question, temperature=temperature, topK=topK, topP=topP,
                                  maxOutputTokens=maxOutputTokens, stopSequences=stopSequences,
                                  verbose=verbose)

In [55]:
list_candidates

[{'content': {'parts': [{'text': "No.\n\nWhile a cytokine storm is associated with severe COVID-19 and contributes to mortality in some cases, it's not a *fundamental* cause of death.  The virus itself, SARS-CoV-2, directly damages lung tissue and other organs, leading to respiratory failure and multi-organ dysfunction.  The cytokine storm is a *consequence* of the infection, a hyperinflammatory response that exacerbates the damage caused by the virus.  Death is often a multifactorial process involving direct viral damage, the immune response (including the cytokine storm), and pre-existing conditions.  Therefore, attributing death solely to the cytokine storm is an oversimplification.\n"}],
   'role': 'model'},
  'finishReason': 'STOP',
  'avgLogprobs': -0.10554480207139168}]

### Resposta total do modelo Gemini 1.5-flash

In [56]:
response = response_candidate(list_candidates, 0)
if response is None:
    response = 'Nothing found'
else:
    response = response.strip()

response

"No.\n\nWhile a cytokine storm is associated with severe COVID-19 and contributes to mortality in some cases, it's not a *fundamental* cause of death.  The virus itself, SARS-CoV-2, directly damages lung tissue and other organs, leading to respiratory failure and multi-organ dysfunction.  The cytokine storm is a *consequence* of the infection, a hyperinflammatory response that exacerbates the damage caused by the virus.  Death is often a multifactorial process involving direct viral damage, the immune response (including the cytokine storm), and pre-existing conditions.  Therefore, attributing death solely to the cytokine storm is an oversimplification."

### Resposta: Sim ou Não

In [58]:
response_yes_no = response.split('.')[0]
response_yes_no

'No'

### Explicação

In [62]:
explanation = ".".join(response.split('.')[1:])
explanation = explanation.replace('\n','')
print(explanation)

While a cytokine storm is associated with severe COVID-19 and contributes to mortality in some cases, it's not a *fundamental* cause of death.  The virus itself, SARS-CoV-2, directly damages lung tissue and other organs, leading to respiratory failure and multi-organ dysfunction.  The cytokine storm is a *consequence* of the infection, a hyperinflammatory response that exacerbates the damage caused by the virus.  Death is often a multifactorial process involving direct viral damage, the immune response (including the cytokine storm), and pre-existing conditions.  Therefore, attributing death solely to the cytokine storm is an oversimplification.
